In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.ensemble import RandomForestRegressor
from prophet import Prophet
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings('ignore')

# Directory path
dir_input = '/kaggle/input/kazakhstan-ai-respa-take-home'
train_df = pd.read_csv(f"{dir_input}/train.csv", parse_dates=['submitted_date'])
test_df = pd.read_csv(f"{dir_input}/test.csv", parse_dates=['week_start','week_end'])
submission_df = pd.read_csv(f"{dir_input}/sample_submission.csv")

# Prepare weekly data
train_df['week_start'] = train_df['submitted_date'].dt.to_period('W').apply(lambda r: r.start_time)
weekly = (
    train_df.groupby(['category','week_start'])['num_papers']
    .sum()
    .reset_index()
    .sort_values(['category','week_start'])
)

def create_time_features(df):
    df = df.copy()
    df['t'] = np.arange(len(df))
    df['month'] = df['week_start'].dt.month
    df['quarter'] = df['week_start'].dt.quarter
    df['sin_woy'] = np.sin(2 * np.pi * df['week_start'].dt.isocalendar().week / 52)
    df['cos_woy'] = np.cos(2 * np.pi * df['week_start'].dt.isocalendar().week / 52)
    df['year'] = df['week_start'].dt.year
    df['day_of_year'] = df['week_start'].dt.dayofyear

    lags = [1, 2, 3, 4, 8, 12, 26, 52]
    windows = [4, 12, 26, 52]

    for lag in lags:
        df[f'lag_{lag}'] = df['num_papers'].shift(lag)
    for w in windows:
        rolled = df['num_papers'].shift(1).rolling(w, min_periods=1)
        df[f'roll_mean_{w}'] = rolled.mean()
        df[f'roll_std_{w}'] = rolled.std()
        df[f'roll_max_{w}'] = rolled.max()
        df[f'roll_min_{w}'] = rolled.min()

    # Fill missing values
    for col in df.columns:
        if col.startswith('lag_'):
            n = int(col.split('_')[1])
            q = 0.005 if n >= 12 else 0.75
            fill = df[col].quantile(q) if not df[col].isna().all() else 0
            df[col] = df[col].fillna(fill)

        elif col.startswith('roll_'):
            w = int(col.split('_')[2])
            q = 0.005 if w >= 12 else 0.75
            fill = df[col].quantile(q) if not df[col].isna().all() else 0
            df[col] = df[col].fillna(fill)

    return df.reset_index(drop=True)

def create_lstm_dataset(X, y, timesteps=4):
    """Create dataset for LSTM model with specified timesteps"""
    Xs, ys = [], []
    for i in range(len(X) - timesteps):
        Xs.append(X[i:(i + timesteps)].values)
        ys.append(y[i + timesteps])
    return np.array(Xs), np.array(ys)

class LSTMModel(nn.Module):
    """PyTorch LSTM model for time series forecasting"""
    def __init__(self, input_size, hidden_size1=64, hidden_size2=32, output_size=1):
        super(LSTMModel, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size1, batch_first=True, return_sequences=True)
        self.dropout1 = nn.Dropout(0.2)
        self.lstm2 = nn.LSTM(hidden_size1, hidden_size2, batch_first=True)
        self.dropout2 = nn.Dropout(0.2)
        self.fc1 = nn.Linear(hidden_size2, 16)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(16, output_size)
        
    def forward(self, x):
        x, _ = self.lstm1(x)
        x = self.dropout1(x)
        x, _ = self.lstm2(x)
        x = self.dropout2(x)
        x = self.relu(self.fc1(x[:, -1, :]))  # Take only the last output
        x = self.fc2(x)
        return x

def train_prophet_model(df):
    """Train Prophet model on dataframe with columns 'week_start' and 'num_papers'"""
    model_data = df[['week_start', 'num_papers']].rename(columns={'week_start': 'ds', 'num_papers': 'y'})
    model = Prophet(yearly_seasonality=True, weekly_seasonality=True)
    model.fit(model_data)
    return model

print('Starting enhanced model training...\n')

# Define expanded model dictionary
test_models = {
    'linear': Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures(1, include_bias=False)), ('lr', LinearRegression())]),
    'ridge1': Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures(1, include_bias=False)), ('ridge', Ridge(alpha=1.0))]),
    'poly2_lr': Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures(2, include_bias=False)), ('lr', LinearRegression())]),
    'ridge2': Pipeline([('scaler', StandardScaler()), ('poly', PolynomialFeatures(2, include_bias=False)), ('ridge', Ridge(alpha=1.0))]),
    'xgboost': xgb.XGBRegressor(
        n_estimators=100,
        learning_rate=0.05,
        max_depth=4,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='reg:squarederror',
        random_state=42
    ),
    'lightgbm': lgb.LGBMRegressor(
        n_estimators=100,
        learning_rate=0.05,
        num_leaves=31,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    ),
    'catboost': cb.CatBoostRegressor(
        iterations=100,
        learning_rate=0.05,
        depth=6,
        loss_function='RMSE',
        random_seed=42,
        verbose=0
    ),
    'random_forest': RandomForestRegressor(
        n_estimators=100,
        max_depth=8,
        min_samples_split=5,
        random_state=42
    )
}

min_scale = 10
val_weeks = 8
results = []
preds = []
all_smape = []
best_models = {}

for cat, group in weekly.groupby('category'):
    print(f"\nProcessing category: {cat}")
    df_feat = create_time_features(group)
    X = df_feat.drop(['category', 'week_start', 'num_papers'], axis=1)
    y = df_feat['num_papers']
    
    # Special handling for short series
    if len(df_feat) <= val_weeks + 4:  # Need at least 4 points after validation for LSTM
        print(f"  Series too short ({len(df_feat)} points), using default model")
        best_model = test_models['poly2_lr']
        best_name = 'poly2_lr'
    else:
        # Standard validation
        df_tr = df_feat.iloc[:-val_weeks]
        df_val = df_feat.iloc[-val_weeks:]
        X_tr = df_tr.drop(['category', 'week_start', 'num_papers'], axis=1)
        y_tr = df_tr['num_papers']
        X_val = df_val.drop(['category', 'week_start', 'num_papers'], axis=1)
        y_val = df_val['num_papers']
        
        # Evaluate standard models
        smape_scores = {}
        for name, model in test_models.items():
            print(f"  Training {name}...")
            m = model
            m.fit(X_tr, y_tr)
            y_pred = m.predict(X_val)
            smape = np.mean(np.abs(y_pred - y_val) / np.maximum(np.abs(y_val), min_scale))
            smape_scores[name] = smape
            results.append({'category': cat, 'model': name, 'smape': smape})
            
        # Try LSTM if we have enough data
        if len(df_tr) >= 8:  # Need at least 8 points for a meaningful LSTM
            print(f"  Training LSTM with PyTorch...")
            try:
                # Set device (CPU or GPU)
                device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
                
                # Prepare LSTM data
                X_lstm_cols = X_tr.columns
                scaler_X = StandardScaler()
                scaler_y = StandardScaler()
                
                X_tr_scaled = pd.DataFrame(scaler_X.fit_transform(X_tr), columns=X_lstm_cols)
                y_tr_scaled = scaler_y.fit_transform(y_tr.values.reshape(-1, 1)).flatten()
                
                X_val_scaled = pd.DataFrame(scaler_X.transform(X_val), columns=X_lstm_cols)
                
                # Create sequences for LSTM
                timesteps = min(4, len(X_tr) // 2)  # Adjust timesteps based on data size
                X_lstm_tr, y_lstm_tr = create_lstm_dataset(X_tr_scaled, y_tr_scaled, timesteps)
                
                if len(X_lstm_tr) > 0:
                    # Convert to PyTorch tensors
                    X_tensor = torch.FloatTensor(X_lstm_tr).to(device)
                    y_tensor = torch.FloatTensor(y_lstm_tr).reshape(-1, 1).to(device)
                    
                    # Create DataLoader
                    batch_size = min(8, len(X_lstm_tr))
                    dataset = TensorDataset(X_tensor, y_tensor)
                    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
                    
                    # Build the model
                    input_size = X_lstm_tr.shape[2]  # Number of features
                    lstm_model = LSTMModel(input_size=input_size).to(device)
                    
                    # Define loss function and optimizer
                    criterion = nn.MSELoss()
                    optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)
                    
                    # Training loop
                    num_epochs = 50
                    best_loss = float('inf')
                    best_model_state = None
                    patience = 5
                    patience_counter = 0
                    
                    for epoch in range(num_epochs):
                        lstm_model.train()
                        epoch_loss = 0
                        
                        for batch_X, batch_y in dataloader:
                            optimizer.zero_grad()
                            outputs = lstm_model(batch_X)
                            loss = criterion(outputs, batch_y)
                            loss.backward()
                            optimizer.step()
                            epoch_loss += loss.item()
                        
                        avg_loss = epoch_loss / len(dataloader)
                        
                        # Early stopping
                        if avg_loss < best_loss:
                            best_loss = avg_loss
                            best_model_state = lstm_model.state_dict().copy()
                            patience_counter = 0
                        else:
                            patience_counter += 1
                            if patience_counter >= patience:
                                break
                    
                    # Load the best model
                    if best_model_state:
                        lstm_model.load_state_dict(best_model_state)
                    
                    # Make predictions
                    lstm_model.eval()
                    lstm_preds = []
                    last_sequence = torch.FloatTensor(X_tr_scaled.values[-timesteps:].reshape(1, timesteps, -1)).to(device)
                    
                    with torch.no_grad():
                        for _ in range(len(X_val)):
                            pred = lstm_model(last_sequence).item()
                            lstm_preds.append(pred)
                            
                            # Update sequence for next prediction
                            next_input = torch.FloatTensor(X_val_scaled.iloc[len(lstm_preds)-1:len(lstm_preds)].values).to(device)
                            last_sequence = torch.cat((last_sequence[:, 1:, :], next_input.view(1, 1, -1)), dim=1)
                    
                    # Inverse transform predictions
                    lstm_preds = scaler_y.inverse_transform(np.array(lstm_preds).reshape(-1, 1)).flatten()
                    
                    # Calculate SMAPE
                    lstm_smape = np.mean(np.abs(lstm_preds - y_val) / np.maximum(np.abs(y_val), min_scale))
                    smape_scores['lstm'] = lstm_smape
                    results.append({'category': cat, 'model': 'lstm', 'smape': lstm_smape})
            except Exception as e:
                print(f"  LSTM error: {str(e)}")
                
        # Try Prophet if we have enough data
        if len(df_tr) >= 10:  # Prophet needs a decent amount of data
            print(f"  Training Prophet...")
            try:
                prophet_model = train_prophet_model(df_tr[['week_start', 'num_papers']])
                
                # Make predictions
                future = prophet_model.make_future_dataframe(periods=val_weeks, freq='W')
                forecast = prophet_model.predict(future)
                prophet_preds = forecast['yhat'].iloc[-val_weeks:].values
                
                # Calculate SMAPE
                prophet_smape = np.mean(np.abs(prophet_preds - y_val) / np.maximum(np.abs(y_val), min_scale))
                smape_scores['prophet'] = prophet_smape
                results.append({'category': cat, 'model': 'prophet', 'smape': prophet_smape})
            except Exception as e:
                print(f"  Prophet error: {str(e)}")
                
        # Try ARIMA if we have enough data
        if len(df_tr) >= 12:  # ARIMA needs a decent amount of data
            print(f"  Training ARIMA...")
            try:
                arima_model = ARIMA(df_tr['num_papers'], order=(1, 1, 1))
                arima_res = arima_model.fit()
                
                # Make predictions
                arima_preds = arima_res.forecast(steps=val_weeks)
                
                # Calculate SMAPE
                arima_smape = np.mean(np.abs(arima_preds - y_val) / np.maximum(np.abs(y_val), min_scale))
                smape_scores['arima'] = arima_smape
                results.append({'category': cat, 'model': 'arima', 'smape': arima_smape})
            except Exception as e:
                print(f"  ARIMA error: {str(e)}")
                
        # Select best model
        best_name = min(smape_scores, key=smape_scores.get)
        print(f"  Best model: {best_name} with sMAPE={smape_scores[best_name]:.4f}")
        all_smape.append(smape_scores[best_name])
        
        # For specialized models, we'll need to store them differently
        if best_name == 'lstm':
            best_models[cat] = {
                'type': 'lstm',
                'model': lstm_model,
                'scaler_X': scaler_X,
                'scaler_y': scaler_y,
                'timesteps': timesteps,
                'X_cols': X_lstm_cols
            }
            continue
        elif best_name == 'prophet':
            best_models[cat] = {
                'type': 'prophet',
                'model': prophet_model
            }
            continue
        elif best_name == 'arima':
            best_models[cat] = {
                'type': 'arima',
                'model': arima_res
            }
            continue
        else:
            best_model = test_models[best_name]
            best_models[cat] = {
                'type': 'standard',
                'model': best_model,
                'name': best_name
            }
    
    # For standard models, fit on all data
    if cat not in best_models:
        best_model = test_models[best_name]
        best_model.fit(X, y)
        best_models[cat] = {
            'type': 'standard',
            'model': best_model,
            'name': best_name
        }

# Make predictions on test data
for cat, cat_test in test_df.groupby('category'):
    print(f"\nGenerating predictions for {cat}...")
    cat_test = cat_test.sort_values('week_id').copy()
    hist = weekly[weekly['category'] == cat].set_index('week_start')['num_papers'].copy()
    test_preds = []
    
    model_info = best_models.get(cat)
    if model_info is None:
        print(f"  No model found for {cat}, using default")
        model_info = {
            'type': 'standard',
            'model': test_models['ridge2'],
            'name': 'ridge2'
        }
    
    model_type = model_info['type']
    
    # Different prediction process depending on model type
    if model_type == 'standard':
        print(f"  Using standard model: {model_info.get('name', 'unknown')}")
        for _, row in cat_test.iterrows():
            wk = row['week_start']
            hist = pd.concat([hist, pd.Series({wk: np.nan})])
            temp = pd.DataFrame({
                'category': cat,
                'week_start': hist.index,
                'num_papers': hist.values
            })
            feat = create_time_features(temp)
            X_test = feat.drop(['category', 'week_start', 'num_papers'], axis=1).iloc[[-1]]
            y_hat = model_info['model'].predict(X_test)[0]
            y_hat = max(0, y_hat)
            test_preds.append(y_hat)
            hist.iloc[-1] = y_hat
            
    elif model_type == 'lstm':
        print(f"  Using PyTorch LSTM model")
        # For LSTM we need a different approach since it uses sequences
        lstm_model = model_info['model']
        scaler_X = model_info['scaler_X']
        scaler_y = model_info['scaler_y']
        timesteps = model_info['timesteps']
        X_cols = model_info['X_cols']
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # Get the historical data with features
        hist_df = weekly[weekly['category'] == cat].copy()
        hist_feat = create_time_features(hist_df)
        X_hist = hist_feat.drop(['category', 'week_start', 'num_papers'], axis=1)
        
        # Scale the data
        X_hist_scaled = pd.DataFrame(scaler_X.transform(X_hist), columns=X_cols)
        
        # Create initial sequence as tensor
        last_sequence = torch.FloatTensor(X_hist_scaled.values[-timesteps:].reshape(1, timesteps, -1)).to(device)
        
        # Put model in evaluation mode
        lstm_model.eval()
        
        for _, row in cat_test.iterrows():
            wk = row['week_start']
            
            # Predict next value
            with torch.no_grad():
                pred_scaled = lstm_model(last_sequence).item()
            
            y_hat = scaler_y.inverse_transform([[pred_scaled]])[0][0]
            y_hat = max(0, y_hat)
            test_preds.append(y_hat)
            
            # Update history
            hist = pd.concat([hist, pd.Series({wk: y_hat})])
            
            # Create new features for this point
            temp = pd.DataFrame({
                'category': cat,
                'week_start': hist.index,
                'num_papers': hist.values
            })
            new_feat = create_time_features(temp)
            new_X = new_feat.drop(['category', 'week_start', 'num_papers'], axis=1).iloc[[-1]]
            
            # Scale new features
            new_X_scaled = scaler_X.transform(new_X)
            
            # Update sequence for next prediction as tensor
            next_input = torch.FloatTensor(new_X_scaled).view(1, 1, -1).to(device)
            last_sequence = torch.cat((last_sequence[:, 1:, :], next_input), dim=1)
            
    elif model_type == 'prophet':
        print(f"  Using Prophet model")
        prophet_model = model_info['model']
        
        # Create future dataframe with test weeks
        future_dates = pd.concat([
            pd.DataFrame({'ds': weekly[weekly['category'] == cat]['week_start']}),
            pd.DataFrame({'ds': cat_test['week_start']})
        ]).drop_duplicates().sort_values('ds')
        
        forecast = prophet_model.predict(future_dates)
        
        # Extract predictions for test weeks
        for _, row in cat_test.iterrows():
            wk = row['week_start']
            y_hat = forecast[forecast['ds'] == wk]['yhat'].values[0]
            y_hat = max(0, y_hat)
            test_preds.append(y_hat)
            hist = pd.concat([hist, pd.Series({wk: y_hat})])
            
    elif model_type == 'arima':
        print(f"  Using ARIMA model")
        arima_res = model_info['model']
        
        # For ARIMA, we need to forecast one step at a time and update
        for _, row in cat_test.iterrows():
            wk = row['week_start']
            y_hat = arima_res.forecast(steps=1)[0]
            y_hat = max(0, y_hat)
            test_preds.append(y_hat)
            
            # Update history and refit model
            hist = pd.concat([hist, pd.Series({wk: y_hat})])
            try:
                # Update ARIMA model with new observation
                arima_model = ARIMA(hist, order=(1, 1, 1))
                arima_res = arima_model.fit()
            except:
                # If refitting fails, keep using the last model
                pass
    
    cat_test['num_papers'] = test_preds
    preds.append(cat_test[['category', 'week_id', 'num_papers']])

# Combine predictions and create submission
if all_smape:
    print(f"\nOverall average sMAPE: {np.mean(all_smape):.4f}")

pred_df = pd.concat(preds)
pred_df['id'] = pred_df['category'] + '__' + pred_df['week_id'].astype(str)
submission = (
    submission_df[['id']]
    .merge(pred_df[['id', 'num_papers']], on='id', how='left')
)
submission['num_papers'] = submission['num_papers'].fillna(0).round().astype(int)
submission.to_csv('submission.csv', index=False)
print("\nSubmission file created: submission.csv")

# Analysis of model performance
model_results = pd.DataFrame(results)
if not model_results.empty:
    print("\nModel performance summary:")
    model_summary = model_results.groupby('model')['smape'].agg(['mean', 'std', 'count']).sort_values('mean')
    print(model_summary)

    print("\nTop model by category:")
    top_models = model_results.loc[model_results.groupby('category')['smape'].idxmin()]
    print(top_models[['category', 'model', 'smape']])